<a href="https://colab.research.google.com/github/AlkaidCheng/GSoC2021_QMLHEP/blob/main/notebooks/TaskV_OpenTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task Statement:

Please comment on quantum computing or quantum machine learning. You can also
comment on one quantum algorithm or one quantum software you are familiar with.
You can also suggest methods you think are good and you would like to work.
Please use your own understanding. Comments copied from the internet will not be
considered.

# Response:


## Comments on Quantum Machine Learning

- **On quantum supremacy**: There is often a misunderstanding that quantum computer is superior in the sense that it can compute problems that classical computer cannot. A correct statement should be that a quantum computer can compute certain type of problem much more efficiently (i.e. better time complexity) than a classical computer. Some classical examples are the Shor's algorithm, Quantum Fourier transform and Grover's algorithm. Such advantage is brought about by the unique features of quantum mechanics that an $N$ qubit system can exist as a superposition of $2^N$ basis states and such states collapse to a single state upon measurement makes it possible to perform parallel computation for a vast number of inputs (ideally $2^N$ of them). So what we should seek for if we are to develop a quantum algorithm is whether it can perform more efficiently than its classical counterpart instead of simply aiming at a better performance.  

- **Challenges of Noisy Intermediate Scale Quantum (NISQ) computers**: Current quantum computing hardware belongs to the family of NISQ with serious problem of noise and is therefore unable to simulate quantum circuits of large depth or with complicated gate expressions. Therefore for any QML algorithms to be of practical use (at least for near future), it has to be simple enough to be run on hardware that does not require much error correction but complex enough to be able to learn anything meaningful.


- **Quantum Data Encoding**: A major source of quantum advantage comes from the high dimensional Hilbert space that qubits lived in which can translated as the feature space of input data. One common scheme of data encoding is to parameterise the rotation angle $\theta$ of gate operations in a quantum circuit by the input feature $\mathbf{x}$ via some encoding function $\theta = f_{enc}(\mathbf{x})$. Such encoding function together with the choice of its associated gate operations determin the expressive power of a quantum circuit for the classical data. In the past year, my colleagues and I have been trying out various encoding functions such as those suggested in https://arxiv.org/abs/1906.10467 but ended up with poor performance. Until recently, we discovered an encoding function of the form $[\sum_{i=1}^k x_i/k]^d$ for a $k$ qubit gate operation with a tunable degree $d$ gives a rather good performance on the quantum support vector machine (QSVM) method. So the take away is that we should be searching for more candidates of encoding functions which can potentially boost the performance of a quantum machine learning algorithm.

## Comments on Quantum Machine Learning Algorithms

- **Qunatum Support Vector Machine (QSVM)**:  This algorithm is good in the sense that it only involves the data encoding circuit and the entire circuit only needs to be executed once for each data point. It also does not involve any randomness in terms of the quantum part of the algorithm, i.e. simulation of final states/quantum kernel value from the data encoding circuit. However, it suffers from a fatal problem that its time complexity grows as $O(N^2)$ with the number of events $N$ due to the need to evaluate the quantum kernel matrix which is $NxN$. This also means that the memory requirement scales by $O(N^2)$. Just like other QML algorithms such as QNN, the time complexity scales with $O(n)$ with number of qubits $n$. However, QSVM suffers from the problem of memory usage which scales as $2^n$ which is way above other QML methods. This essentially makes it not applicable to large dataset. Approximation methods for the kernel matrix will be needed in order to make this method viable.

- **Quantum Kernel method vs Quantum Neural Network**: It has been said that the future of quantum machine learning will likely be between quantum kernel method and quantum neural networks. Some researchers (e.g. Maria Schuld https://arxiv.org/abs/2101.11020) believed that quantum machine learning methods are essentially identical to kernel methods. From a personal standpoint, QNN should be a much more sought after direction due to its vast varieties and we have seen great success from classical neural networks. Currently, quantum analougue of classical neural networks such as QGAN, QCNN, QGNN, QVAE have seen very rapid development and it would only make sense we also try to apply these novel methods to actual scientific applications. 

- **Quantum Generative Adversarial Network (QGAN)**: For this algorithm to actually work we need to assume that a quantum circuit can be effectively constructed to represent any probability distribution of some input data. Given the high dimensional Hilbert space that qubit states lived in, it is convincing that we can construct circuits to represent arbitrary probability distribution. However, it remains difficult to find a construction that can efficiently explore the entire phase space available to a given number of qubits that does not require a lot of parameterised gate operations. So our key to develop a good QGAN would be to find out a good PQC design that can represents many probability distribution well. Another issue for QGAN is using it for classification. Natively QGAN is not used for classification since it can only distinguish between a true input and an input that comes from noise. So even for binary classification problem, a QGAN trained on only one class of data would not be able to effectively distingish data between two classes since it has no knowledge about the other class. Therefore, we will need some clever construction of possibly multiple generators and discrminators in order to make it useful for classification problem. So this will be one direction to work on.

## Comments on the `cirq` and `tensorflow_quantum` packages

- **Gradient Calculation** and **Parameter Resolving**: In the native way of implementing quantum neural network layers (called PQC) using `tensorflow_quantum` (at least according to their tutorials), the input to the PQC layer is a data encoding quantum circuit that has the classical data already encoded into the parameters of the circuit gate operations. This circuit is converted into a tensorflow string and passed as an input to the PQC layer. However, this causes problem in **gradient calculation** that involves output from multiple neural networks, e.g. generator and discrminator for Quantum GAN. Since the generator output is a tensor of the classical input which needs to be encoded into a data encoding circuit and then to a tensorflow string, in this process the **gradient information with respect to the generator weights is lost and can not be passed to the discriminator (i.e. no back propagation)**. To circumvent this I implemented my custom PQC layer in my own package `quple` which takes in tensor with classical data instead of tensor with the data encoding circuit in the string representation. This is achieved by combining the data encoding circuit and the parameterised circuit as a whole and resolve the symbol values of both circuits all together. However, there is a current limitation from `tensorflow_quntum` that it cannot resolve symbol expression other than simple scalar multiplication which greatly limits the complexity of the data encoding circuit. The relevant code that causes the problem is: https://github.com/tensorflow/quantum/blob/master/tensorflow_quantum/core/serialize/serializer.py#L43

- **Support for higher order differentiation**: `tensorflow_quantum` does not support second order differentiation of quantum layer therefore we can not use models that require second derivatives, for instance the **Wasserstein GAN with gradient penalty** (WGAN-GP). The relevant discussion can be found here: https://github.com/tensorflow/quantum/issues/285.

- **Translation to QASM code**:`cirq` does not support native translation of quantum circuit into a raw representation in terms of the universal QASM language. This conversion, however, is supported by IBM Qiskit. The importance of having a universal representation of a circuit makes it a portable component across different libraries and hence different hardware devices.

## Comments on Important Research Related to QML

- **Circuit Differentiators**: Currently `tensorflow_quantum` support a variety of differentors for automatic differentiation of PQCs, including parameter shift, adjoint differentor and forward difference. It will be important for the future to have more efficient differentiators to cut down simulation time.

- **Quantum Optimizers**: One of the major obstacle of developing a fully quantum algorithm is the lack of a quantum optimization algorithm. Currently there are many promising reasearch such as quantum approximate optimization algorithm (QAOA) (https://arxiv.org/abs/1411.4028) that can achieve similar results. However most quantum optimizers currently available are only suitable for a very narrow type of problems and not for optimization of a general function such as those used in neural networks. 

- **Circuit Descriptors**: Often the times we don't have a good idea when choosing what data encoding circuit or parameterised quantum circuit to use for a QML model since there are countless variations of circuits that can be constructed. Simply trial and error to choose the best circuit for a problem is not only time consuming but also gives poor results most of the time. Therefore it is important to have a standard measure on some key properties of a circuit that can tell us whether it is good for a specific purpose. Such properties are also called circuit descriptors. For example in this paper https://arxiv.org/abs/1905.10876, the authors introduced an expressibility measure and an entanglement measure for parameterised quantum ciruits (PQC) which helps quantify how well a PQC can explore the phase space and how entangled are the qubits. There are also other descriptors such as fidelity and entropy that are available in the `cirq` library. Therefore, it would be great to implement more circuit descriptors from existing researches to help the community come up with better ways to evaluate their quantum circuit design.





$test$